In [ ]:
import pandas as pd

In [ ]:
import numpy as np


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Carregar os dados do arquivo CSV
df = pd.read_csv('dadosacoes.csv')
df = pd.read_csv('dadosacoes_limpos.csv')
 
# Limpar os nomes das colunas, removendo espaços extras e caracteres indesejados
df.columns = df.columns.str.strip()  # Remove espaços extras à esquerda e à direita
 
# Remover símbolos 'R$', e qualquer outro sufixo que não faça parte do nome da coluna
df.columns = df.columns.str.replace(r' R\$', '', regex=True)  # Remove o símbolo 'R$'
df.columns = df.columns.str.replace(r'\s*//.*', '', regex=True)  # Remove qualquer texto após '//' (como ' -(Bilhões)')
df.columns = df.columns.str.replace(r'\s*-\s*', '_', regex=True)  # Substitui o '-' por um '_'
df.columns = df.columns.str.replace(r'\s+', '_', regex=True)  # Substitui espaços em branco por '_'
 
# Verificar os nomes das colunas após a limpeza
print(df.columns)
 
# Garantir que as colunas numéricas estão no tipo correto
df['preco_acao'] = pd.to_numeric(df['preco_acao'], errors='coerce')
df['qtde_cotas'] = pd.to_numeric(df['qtde_cotas'], errors='coerce')
df['valor_de_mercado'] = pd.to_numeric(df['valor_de_mercado'], errors='coerce')  # Agora com o nome correto
 
# Verificar os tipos de dados das colunas
print(df.dtypes)
 
# Verificar valores ausentes
print(df.isnull().sum())
 
# Verificar duplicatas
print(df.duplicated().sum())
 
# Verificar os primeiros registros
print(df.head())
 
# Salvar o DataFrame limpo
df.to_csv('dadosacoes_limpos.csv', index=False)
 
# Carregar os dados (supondo que o DataFrame df já esteja limpo e com as colunas renomeadas corretamente)
# df = pd.read_csv('dadosacoes.csv')  # Descomente se precisar carregar o CSV
 
# a) Plotar box plot do preço da ação para cada ativo para verificar outliers
 
plt.figure(figsize=(12, 6))
sns.boxplot(x='nome_acao', y='preco_acao', data=df)
plt.title('Box Plot do Preço da Ação por Ativo')
plt.xticks(rotation=90)  # Rotacionar os nomes das ações para melhor visualização
plt.xlabel('Nome da Ação')
plt.ylabel('Preço da Ação (R$)')
plt.show()
 
# b) Plotar box plot do valor de mercado da empresa para cada ativo para verificar outliers
 
plt.figure(figsize=(12, 6))
sns.boxplot(x='nome_acao', y='valor_de_mercado', data=df)
plt.title('Box Plot do Valor de Mercado por Ativo')
plt.xticks(rotation=90)  # Rotacionar os nomes das ações para melhor visualização
plt.xlabel('Nome da Ação')
plt.ylabel('Valor de Mercado (Bilhões R$)')
plt.show()
 
# c) Exibir informações do conjunto de dados (df.info()) e estatísticas descritivas (df.describe())
 
# Informações gerais sobre o DataFrame
print("Informações gerais do DataFrame:")
print(df.info())
 
# Estatísticas descritivas do DataFrame
print("\nEstatísticas descritivas do DataFrame:")
print(df.describe())
 
 
print(type(df))
 
# Recarregar o CSV, caso df tenha sido sobrescrito
df = pd.read_csv('dadosacoes_limpos.csv')
 
# Verificar o tipo de df para garantir que é um DataFrame
print(f'Tipo de df: {type(df)}')
 
# Exibir as primeiras linhas do DataFrame
print("\nPrimeiras linhas do DataFrame:")
print(df.head())
 
# Lidar com valores ausentes
# Verificar se há valores ausentes no dataset
print("\nValores ausentes por coluna:")
print(df.isnull().sum())  # Isso deve funcionar se df for um DataFrame
 
# Preencher valores ausentes com a mediana (para as colunas numéricas)
df['preco_acao'] = df['preco_acao'].fillna(df['preco_acao'].median())
df['qtde_cotas'] = df['qtde_cotas'].fillna(df['qtde_cotas'].median())
df['valor_de_mercado'] = df['valor_de_mercado'].fillna(df['valor_de_mercado'].median())
 
# Codificar variáveis categóricas
# Transformar a coluna 'nome_acao' em variáveis dummy (binárias)
df = pd.get_dummies(df, columns=['nome_acao'], drop_first=True)
 
# Verificar o DataFrame após a codificação
print("\nDataFrame após a codificação das variáveis categóricas:")
print(df.head())
 
# Normalizar os dados
scaler = StandardScaler()
df_scaled = df.copy()  # Criar uma cópia para manter os dados originais
 
# Normalizar as colunas numéricas
df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']] = scaler.fit_transform(df[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# Verificar o DataFrame normalizado
print("\nDataFrame normalizado:")
print(df_scaled.head())
 
# Carregar os dados (supondo que df já tenha sido limpo)
df = pd.read_csv('dadosacoes_limpos.csv')
 
# Normalizar os dados para o K-means (com base nas variáveis numéricas)
scaler = StandardScaler()
df_scaled = df.copy()  # Criar uma cópia para manter os dados originais
 
# Normalizar as colunas numéricas
df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']] = scaler.fit_transform(df[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# ---- Etapa a: Aplicar K-means com n_clusters=4 ----
print("\n--- Etapa a: K-means com n_clusters=4 ---")
# Aplicando K-means com n_clusters = 4
kmeans_4 = KMeans(n_clusters=4, random_state=42)
df_scaled['cluster_4'] = kmeans_4.fit_predict(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# Exibir os primeiros registros com os clusters atribuídos
print("\nPrimeiras linhas com o agrupamento de 4 clusters:")
print(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado', 'cluster_4']].head())
 
# ---- Etapa b: Alterar o número de clusters para 5 ----
print("\n--- Etapa b: Alterar o número de clusters para 5 ---")
# Aplicando K-means com n_clusters = 5
kmeans_5 = KMeans(n_clusters=5, random_state=42)
df_scaled['cluster_5'] = kmeans_5.fit_predict(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# Exibir os primeiros registros com os clusters atribuídos
print("\nPrimeiras linhas com o agrupamento de 5 clusters:")
print(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado', 'cluster_5']].head())
 
# ---- Etapa c: Plotar o gráfico do cotovelo para n variando de 1 a 8 clusters ----
print("\n--- Etapa c: Gráfico do Cotovelo ---")
# Método do Cotovelo: calcular a inércia para diferentes números de clusters
inercia = []
for k in range(1, 9):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
    inercia.append(kmeans.inertia_)
 
# Plotando o gráfico do cotovelo
plt.figure(figsize=(8, 6))
plt.plot(range(1, 9), inercia, marker='o')
plt.title('Método do Cotovelo para Escolha de k')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.xticks(range(1, 9))
plt.grid(True)
plt.show()
 
# ---- Etapa d: Plotar o gráfico da silhueta ----
print("\n--- Etapa d: Gráfico da Silhueta ---")
# Calcular e plotar a silhueta para diferentes números de clusters
silhouettes = []
for k in range(2, 9):  # Não faz sentido calcular para 1 cluster
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
    score = silhouette_score(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']], kmeans.labels_)
    silhouettes.append(score)
 
# Plotando o gráfico da silhueta
plt.figure(figsize=(8, 6))
plt.plot(range(2, 9), silhouettes, marker='o')
plt.title('Análise de Silhueta para Diferentes Valores de k')
plt.xlabel('Número de Clusters')
plt.ylabel('Pontuação de Silhueta')
plt.xticks(range(2, 9))
plt.grid(True)
plt.show()
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
 
# Carregar os dados (supondo que df já tenha sido limpo)
df = pd.read_csv('dadosacoes_limpos.csv')
 
# Normalizar os dados para o K-means (com base nas variáveis numéricas)
scaler = StandardScaler()
df_scaled = df.copy()  # Criar uma cópia para manter os dados originais
 
# Normalizar as colunas numéricas
df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']] = scaler.fit_transform(df[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# Aplicar o K-means com n_clusters=4 (Você pode alterar o número de clusters conforme necessário)
kmeans = KMeans(n_clusters=4, random_state=42)
df_scaled['cluster'] = kmeans.fit_predict(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# ---- a) Visualização 2D dos clusters formados ----
# Redução de dimensionalidade para 2D usando PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_scaled[['preco_acao', 'qtde_cotas', 'valor_de_mercado']])
 
# Criar o gráfico 2D
plt.figure(figsize=(10, 6))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=df_scaled['cluster'], palette='Set1', s=100, alpha=0.7, edgecolor='w', marker='o')
 
# Títulos e labels
plt.title('Visualização 2D dos Clusters Formados (PCA)', fontsize=16)
plt.xlabel('Componente Principal 1', fontsize=12)
plt.ylabel('Componente Principal 2', fontsize=12)
plt.legend(title='Cluster', loc='best', fontsize=12)
plt.show()
 
# ---- b) Visualização 3D dos clusters formados ----
# Criar o gráfico 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
 
# Plotando os pontos 3D com a cor representando o cluster
ax.scatter(df_scaled['preco_acao'], df_scaled['qtde_cotas'], df_scaled['valor_de_mercado'], c=df_scaled['cluster'], cmap='Set1', s=100)
 
# Títulos e labels
ax.set_title('Visualização 3D dos Clusters Formados', fontsize=16)
ax.set_xlabel('Preço da Ação (Normalizado)', fontsize=12)
ax.set_ylabel('Quantidade de Cotas (Normalizado)', fontsize=12)
ax.set_zlabel('Valor de Mercado (Normalizado)', fontsize=12)
plt.show()